In [37]:
from PIL import Image as PILImage
import io
import base64
import json
import os
from azure.iot.device import IoTHubDeviceClient
from azure.iot.device.exceptions import ConnectionFailedError
import time

# Azure IoT Hub configuration with device "snower"
with open("connect_string.txt", "r") as f:
    CONNECTION_STRING = f.read().strip('"')


#let's try taking screenshots, formatting them and sending them to the cloud
# we will not use ROS in this case
import pyscreenshot as ImageGrab

def resize_image(image, max_size=(480, 480)):
    """Resize image to fit within max_size without exceeding it"""
    image.thumbnail(max_size, PILImage.LANCZOS)
    return image


In [38]:
try:
    if client:
        client.shutdown()
except:
    pass

In [39]:
try:
    client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)
    print("Connected to Azure IoT Hub.")
except ConnectionFailedError:
    print("Failed to connect to Azure IoT Hub.")
    client = None

Connected to Azure IoT Hub.


In [40]:
# take a screenshot
img = ImageGrab.grab()

# Resize image if needed
resized_image = resize_image(img)

# Convert the image to a base64 string
buffered = io.BytesIO()
resized_image.save(buffered, format="JPEG")
image_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')

# Create JSON payload
payload = json.dumps({
    "image_data": image_base64
})

# Check payload size
print("Payload size: ", len(payload.encode('utf-8')))

Payload size:  15714


In [41]:
payload

'{"image_data": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEOAeADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD0GPT9Ra4Mmwtbsg2KF5HvnvTl03UwjhoiWP3CExj681eul1VGsZLG3SZcxh98u3Ym3njPfnPXtWrd3FsiASzvGA2MqSCfyrolUeg1oc4NN1PyiDEfMzwdnGKaumavxujU+uIz/jW6l7YLIsn22VvYsSD

In [42]:
client.send_message(payload)
print("Message sent to Azure IoT Hub.")
client.shutdown()

Message sent to Azure IoT Hub.
